## Heart Disease
El conjunto de datos "Heart Disease" contiene información médica sobre pacientes que se sometieron a pruebas para detectar enfermedades cardíacas. Estos datos fueron recopilados en un estudio realizado en el Instituto de Aprendizaje Automático de la Universidad de California, Irvine (UCI).

Atributos:
<ul>
<li>age: Edad del paciente (numérico).</li>
<li>sex: Género del paciente (0 = mujer, 1 = hombre).</li>
<li>cp: Tipo de dolor en el pecho (0 = típico angina, 1 = angina atípica, 2 = dolor no anginal, 3 = asintomático).</li>
<li>trestbps: Presión arterial en reposo (en mm Hg) (numérico).</li>
<li>chol: Colesterol sérico en mg/dl (numérico).</li>
<li>fbs: Nivel de azúcar en sangre en ayunas > 120 mg/dl (0 = falso, 1 = verdadero).</li>
<li>restecg: Resultados electrocardiográficos en reposo (0 = normal, 1 = con anormalidad de la onda ST-T, 2 = hipertrofia ventricular izquierda probable o definitiva).</li>
<li>thalach: Frecuencia cardíaca máxima alcanzada (numérico).</li>
<li>exang: Angina inducida por ejercicio (0 = no, 1 = sí).</li>
<li>oldpeak: Depresión del segmento ST inducida por el ejercicio en relación con el descanso (numérico).</li>
<li>slope: Pendiente del segmento ST de ejercicio máximo (0 = ascendente, 1 = plano, 2 = descendente).</li>
<li>ca: Número de vasos principales (0-3) coloreados por fluoroscopia (numérico).</li>
<li>thal: Enfermedad de Talasemia (3 = normal, 6 = defecto fijo, 7 = defecto reversible).</li>
<li>target: clase objetivo. Presencia de enfermedad cardíaca (0 = no, 1 = sí).</li>
</ul>

Se puede obtener en https://www.kaggle.com/datasets/johnsmith88/heart-disease-dataset

In [115]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.metrics import AUC
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import accuracy_score

## Carga de datos y preparación

In [116]:
# Cargar el conjunto de datos
data = pd.read_csv("heart.csv", header=0)

In [117]:
data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1


In [118]:
# Identificar filas con valores nulos
rows_with_nans = data[data.isnull().any(axis=1)]
rows_with_nans

# Reemplazar valores faltantes
data = data.dropna()

In [119]:
data.shape

(1025, 14)

In [120]:
# Mostrar los valores diferentes de la clase target
unique_values = data['target'].unique()
print(f"Valores diferentes en la clase 'target': {unique_values}")
# Transformar la variable de salida (target) en binaria
data['target'] = data['target'].apply(lambda x: 1 if x > 0 else 0)

Valores diferentes en la clase 'target': [0 1]


In [121]:
# División de características y etiquetas
X = data.drop('target', axis=1)
y = data['target']

In [122]:
# División en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [123]:
# Antes de la tranformacion
X_train

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
835,49,1,2,118,149,0,0,126,0,0.8,2,3,2
137,64,0,0,180,325,0,1,154,1,0.0,2,0,2
534,54,0,2,108,267,0,0,167,0,0.0,2,0,2
495,59,1,0,135,234,0,1,161,0,0.5,1,0,3
244,51,1,2,125,245,1,0,166,0,2.4,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,41,1,2,130,214,0,0,168,0,2.0,1,0,2
71,61,1,0,140,207,0,0,138,1,1.9,2,1,3
106,51,1,0,140,299,0,1,173,1,1.6,2,0,3
270,43,1,0,110,211,0,1,161,0,0.0,2,0,3


In [124]:
X_train.shape

(820, 13)

In [125]:
# Preprocesamiento de variables numéricas y categóricas
numeric_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
categorical_features = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

# Aplicar diferentes transformaciones a diferentes columnas de un DataFrame
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

In [126]:
# Preprocesar los datos
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [127]:
# Despues de la transformacion
X_train

array([[-0.58584022, -0.77945357, -1.93503098, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.05147737,  2.74173173,  1.61063407, ...,  0.        ,
         1.        ,  0.        ],
       [-0.04006769, -1.34738668,  0.44217627, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.36753121,  0.46999928,  1.08684264, ...,  0.        ,
         0.        ,  1.        ],
       [-1.24076726, -1.23380006, -0.68598988, ...,  0.        ,
         0.        ,  1.        ],
       [-0.2583767 , -1.12021343, -0.30321922, ...,  0.        ,
         1.        ,  0.        ]])

In [128]:
# Obtener el número de características después del preprocesamiento. 30 porque aplica onehot
input_shape = X_train.shape[1]
input_shape

30

## Modelo básico

In [129]:
# Construir el modelo
# Sequential es la clase en Keras que permite crear un modelo secuencial, lo que significa que las capas del modelo se apilan una tras otra en un solo flujo lineal
# Dense es una clase que representa una capa totalmente conectada
# Units es el número de neuronas en la capa que es el primer parametro
# Activation: la función de activación que se aplica a la salida de las neuronas en esta capa
# -relu (Rectified Linear Unit): max(0, x). Es una función no lineal 
# -sigmoid: 1 / (1 + exp(-x)). Es útil para la salida de una red neuronal que realiza clasificación binaria, ya que transforma la salida en un valor entre 0 y 1.
# -softmax': Específica para problemas de clasificación multiclase. Convierte las salidas en probabilidades que suman a 1.
# -Para problemas de regresión, no se suele utilizar ninguna función de activación específica.
# input_shape: La forma de los datos de entrada. Solo se especifica en la primera capa del modelo. Se especifica en forma de tupla
# Se utilizan las potencias de 2 debido a las eficiencias computacionales que pueden brindar, especialmente en hardware moderno como GPUs.
# Respecto a las funciones de activación, no es obligatorio utilizar la misma función de activación en todas las capas
model = Sequential([
    # Primera capa con 64 neuronas y activación ReLU
    Dense(64, activation='relu', input_shape=(input_shape,)),  
    # Segunda capa con 32 neuronas y activación ReLU
    Dense(32, activation='relu'), 
    # Capa de salida con activación Sigmoid para clasificación binaria
    Dense(1, activation='sigmoid') 
])

In [130]:
# Compilar el modelo
#optimizer: especifica el algoritmo que se utilizará para ajustar los pesos del modelo durante el entrenamiento. Ejemplos comunes son adam, sgd, rmsprop
# loss: es la función de pérdida que se utilizará para evaluar qué tan bien el modelo está realizando la tarea de aprendizaje. La función de pérdida calcula la diferencia entre las predicciones del modelo y las etiquetas reales. Se utiliza binary_crossentropy para problemas de clasificacion binaria
# metrics: son las métricas que se evaluarán durante el entrenamiento y la evaluación del modelo. 
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [131]:
# Resumen del modelo
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_51 (Dense)            (None, 64)                1984      
                                                                 
 dense_52 (Dense)            (None, 32)                2080      
                                                                 
 dense_53 (Dense)            (None, 1)                 33        
                                                                 
Total params: 4097 (16.00 KB)
Trainable params: 4097 (16.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [132]:
# Entrenar el modelo
# epochs: define el número de veces que el algoritmo de entrenamiento recorrerá todo el conjunto de datos de entrenamiento. Cada pasada completa del conjunto de datos se llama una "época". 
# batch_size: define el número de muestras que se procesarán antes de actualizar los parámetros del modelo.
# validation_split: es la fracción del conjunto de datos de entrenamiento que se reservará para validar el modelo durante el entrenamiento. Este conjunto de datos de validación se utiliza para evaluar el rendimiento del modelo en datos no vistos durante el entrenamiento
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
21/21 [==============================] - 1s 6ms/step - loss: 0.6083 - accuracy: 0.7439 - val_loss: 0.5728 - val_accuracy: 0.7622
Epoch 2/50
21/21 [==============================] - 0s 2ms/step - loss: 0.4908 - accuracy: 0.8049 - val_loss: 0.5002 - val_accuracy: 0.7866
Epoch 3/50
21/21 [==============================] - 0s 2ms/step - loss: 0.4033 - accuracy: 0.8460 - val_loss: 0.4398 - val_accuracy: 0.7927
Epoch 4/50
21/21 [==============================] - 0s 2ms/step - loss: 0.3436 - accuracy: 0.8506 - val_loss: 0.3949 - val_accuracy: 0.8354
Epoch 5/50
21/21 [==============================] - 0s 2ms/step - loss: 0.3098 - accuracy: 0.8735 - val_loss: 0.3695 - val_accuracy: 0.8537
Epoch 6/50
21/21 [==============================] - 0s 2ms/step - loss: 0.2853 - accuracy: 0.8872 - val_loss: 0.3538 - val_accuracy: 0.8598
Epoch 7/50
21/21 [==============================] - 0s 2ms/step - loss: 0.2659 - accuracy: 0.8918 - val_loss: 0.3466 - val_accuracy: 0.8537
Epoch 8/50
21/21 [==

In [133]:
# Evaluar el modelo
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc}')


7/7 [==============================] - 0s 2ms/step - loss: 0.1712 - accuracy: 0.9512
Test Accuracy: 0.9512194991111755


## Red neuronal con más capas

In [134]:
# Crear un modelo secuencial
model = Sequential([
    Dense(64, activation='relu', input_shape=(input_shape,)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [135]:
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [136]:
# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
21/21 [==============================] - 1s 6ms/step - loss: 0.6852 - accuracy: 0.4878 - val_loss: 0.6691 - val_accuracy: 0.4756
Epoch 2/50
21/21 [==============================] - 0s 2ms/step - loss: 0.6179 - accuracy: 0.5503 - val_loss: 0.6278 - val_accuracy: 0.5732
Epoch 3/50
21/21 [==============================] - 0s 2ms/step - loss: 0.5512 - accuracy: 0.7546 - val_loss: 0.5657 - val_accuracy: 0.7378
Epoch 4/50
21/21 [==============================] - 0s 2ms/step - loss: 0.5023 - accuracy: 0.8506 - val_loss: 0.5317 - val_accuracy: 0.7988
Epoch 5/50
21/21 [==============================] - 0s 2ms/step - loss: 0.4705 - accuracy: 0.8537 - val_loss: 0.5126 - val_accuracy: 0.8841
Epoch 6/50
21/21 [==============================] - 0s 2ms/step - loss: 0.4453 - accuracy: 0.8902 - val_loss: 0.5047 - val_accuracy: 0.8537
Epoch 7/50
21/21 [==============================] - 0s 2ms/step - loss: 0.4249 - accuracy: 0.9207 - val_loss: 0.5004 - val_accuracy: 0.8720
Epoch 8/50
21/21 [==

In [137]:
# Evaluar el modelo
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc}')

7/7 [==============================] - 0s 2ms/step - loss: 0.3691 - accuracy: 0.9561
Test Accuracy: 0.9560975432395935


## Red neuronal con parámetros de kernel

In [ ]:
# Crear un modelo secuencial
# kernel_initializer: Este parámetro especifica el método para inicializar los pesos de la red neuronal. Valores comunes son 'glorot_uniform', 'he_normal',
# kernel_regularizer: Se utiliza para aplicar penalizaciones a los pesos de la red neuronal durante el entrenamiento
# dropout: Esta capa se puede agregar entre capas densas para ayudar a prevenir el sobreajuste. dropout apaga aleatoriamente un porcentaje de unidades (neuronas) durante el entrenamiento
model = Sequential([
    # para aplicar una regularización L2 a los pesos de la primera capa densa
    Dense(64, activation='relu', input_shape=(input_shape,), kernel_regularizer=regularizers.l2(0.01)), 
    # Se agrega una capa de Dropout con una tasa del 50% para ayudar a prevenir el sobreajuste
    Dropout(0.5), 
    # para inicializar los pesos de la segunda capa densa con el método 'he_normal'
    Dense(32, activation='relu', kernel_initializer='he_normal'), 
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compilar el modelo
# Compilamos el modelo con el optimizador RMSprop, la función de pérdida mean_squared_error, y la métrica AUC.
model.compile(optimizer=RMSprop(), loss='mean_squared_error', metrics=[AUC()])

In [ ]:
# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_split=0.3)


In [ ]:
# Evaluar el modelo
test_loss, test_auc = model.evaluate(X_test, y_test)
print(f'Test AUC: {test_auc}')

## Red neuronal con parámetros de bias

In [ ]:
# Crear un modelo secuencial
# bias_initializer: Similar a kernel_initializer, este parámetro especifica el método para inicializar los sesgos de la red neuronal.
# bias_regularizer: Similar a kernel_regularizer, pero se aplica a los sesgos de la red neuronal.
# activity_regularizer: Se utiliza para aplicar penalizaciones a la actividad de las neuronas de la red neuronal durante el entrenamiento.
model = Sequential([
    Dense(64, activation='relu', input_shape=(input_shape,),
          kernel_regularizer=regularizers.l2(0.01),
          # bias_initializer para inicializar los sesgos de las capas densas. En la primera capa, los sesgos se inicializan como ceros
          bias_initializer='zeros', 
          # bias_regularizer para aplicar una regularización L2 a los sesgos de las capas densas. Se aplica regularización L2 con un factor de penalización de 0.01 en ambas capas.
          bias_regularizer=regularizers.l2(0.01),
          # activity_regularizer para aplicar una regularización L2 a la actividad de las neuronas en las capas densas. También se aplica regularización L2 con un factor de penalización de 0.01 en ambas capas.
          activity_regularizer=regularizers.l2(0.01)),
    Dropout(0.5),
    Dense(32, activation='relu', kernel_initializer='he_normal',
          bias_initializer='ones', # en la segunda capa se inicializan como unos
          bias_regularizer=regularizers.l2(0.01),
          activity_regularizer=regularizers.l2(0.01)),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compilar el modelo
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_split=0.3)

In [ ]:
# Evaluar el modelo
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc}')

## Red neuronal con más capas

In [ ]:
# Crear un modelo secuencial
# BatchNormalization normaliza la activación de cada neurona en la capa anterior, lo que ayuda a estabilizar y acelerar el proceso de entrenamiento.
model = Sequential([
    Dense(128, activation='relu', input_shape=(input_shape,),
          kernel_regularizer=regularizers.l2(0.01),
          bias_initializer='zeros',
          bias_regularizer=regularizers.l2(0.01),
          activity_regularizer=regularizers.l2(0.01)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu', kernel_initializer='he_normal',
          bias_initializer='ones',
          bias_regularizer=regularizers.l2(0.01),
          activity_regularizer=regularizers.l2(0.01)),
    Dropout(0.5),
    Dense(32, activation='relu',
          kernel_regularizer=regularizers.l2(0.01),
          bias_initializer='zeros',
          bias_regularizer=regularizers.l2(0.01),
          activity_regularizer=regularizers.l2(0.01)),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compilar el modelo
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_split=0.3)

In [ ]:
# Evaluar el modelo
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc}')

## Callbacks

In [ ]:
# Configurar el optimizador con una tasa de aprendizaje específica
# Tasa de Aprendizaje (Learning Rate): Controla la magnitud de las actualizaciones de los pesos durante el entrenamiento
optimizer = Adam(learning_rate=0.001)

In [ ]:
# Construir la arquitectura de la red neuronal
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compilar el modelo
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Definir callbacks
# Callbacks: Funciones que se llaman en ciertos puntos durante el entrenamiento, como al final de cada época. Pueden ser útiles para realizar tareas como el almacenamiento de checkpoints, el ajuste dinámico de la tasa de aprendizaje, etc.
# EarlyStopping para detener el entrenamiento si la pérdida en el conjunto de validación deja de disminuir después de cierto número de épocas (patience), y ModelCheckpoint para guardar el modelo con la menor pérdida en el conjunto de validación durante el entrenamiento.
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
]

In [ ]:
# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=callbacks)


In [ ]:
# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

## Validación cruzada

In [ ]:
# Normalizar características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Definir la arquitectura del modelo
def create_model():
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    return model

In [ ]:
# Inicializar KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Inicializar lista para almacenar resultados de precisión
accuracy_scores = []

In [ ]:
# Realizar validación cruzada
for train_index, test_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Crear y compilar el modelo
    model = create_model()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Entrenar el modelo
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)
    
    # Evaluar el modelo en los datos de prueba
    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

In [ ]:
# Calcular la precisión media de la validación cruzada
mean_accuracy = np.mean(accuracy_scores)
print(f'Accuracy (Cross-Validation): {mean_accuracy}')